In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df=pd.read_csv('/content/PS_20174392719_1491204439457_log.csv')

In [4]:
#importing required libraries
import pandas as pd
import numpy as np
import plotly.express as px


The below column reference:

- step: represents a unit of time where 1 step equals 1 hour
- type: type of online transaction
- amount: the amount of the transaction
- nameOrig: customer starting the transaction
- oldbalanceOrg: balance before the transaction
- newbalanceOrig: balance after the transaction
- nameDest: recipient of the transaction
- oldbalanceDest: initial balance of recipient before the transaction
- newbalanceDest: the new balance of recipient after the transaction
- isFraud: fraud transaction

In [5]:
data = pd.read_csv('/content/PS_20174392719_1491204439457_log.csv')
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


# Study and preparing data

In [6]:
#Cheking for Null values

data.isnull().sum()  #does not have any null values

step              0
type              0
amount            0
nameOrig          1
oldbalanceOrg     1
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64

In [7]:
data.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,274057.000000,2.740570e+05,2.740560e+05,2.740560e+05,2.740560e+05,2.740560e+05,274056.000000,274056.0
mean,11.104610,1.778036e+05,8.827004e+05,9.013853e+05,9.659261e+05,1.195209e+06,0.000617,0.0
std,2.513116,3.087131e+05,2.818221e+06,2.855856e+06,2.364294e+06,2.604344e+06,0.024825,0.0
min,1.000000,3.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
25%,9.000000,1.257283e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
50%,11.000000,7.601030e+04,1.862850e+04,0.000000e+00,7.576453e+04,1.762851e+05,0.000000,0.0
75%,13.000000,2.315638e+05,1.838186e+05,2.220743e+05,8.332276e+05,1.232287e+06,0.000000,0.0
max,15.000000,1.000000e+07,3.893942e+07,3.894623e+07,4.133844e+07,4.138365e+07,1.000000,0.0


# type of transaction present in Dataset

In [8]:
type_transaction = data["type"].value_counts()
transaction = type_transaction.index
quantity = type_transaction.values

# plotting pie chart
fig = px.pie(data,
            values = quantity,
            names = transaction,
            hole = 0.4,
            title = "Distribution of Transaction Type")
fig.show()

# Checking for co relataion between the features with isFraud column

In [9]:
corr = data.corr()
corr["isFraud"].sort_values(ascending=False)

<ipython-input-9-745150298315>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



isFraud           1.000000
amount            0.048907
oldbalanceOrg    -0.000935
newbalanceDest   -0.004107
newbalanceOrig   -0.007667
oldbalanceDest   -0.007737
step             -0.031857
isFlaggedFraud         NaN
Name: isFraud, dtype: float64

# transform the categorical features into numerical.

In [10]:
data["type"] = data["type"].map({"CASH_OUT": 1,
                                 "PAYMENT": 2,
                                 "CASH_IN": 3,
                                 "TRANSFER": 4,
                                 "DEBIT": 5})
data["isFraud"] = data["isFraud"].map({0: "No Fraud", 1: "Fraud"})
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,2,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,No Fraud,0.0
1,1,2,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,No Fraud,0.0
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,Fraud,0.0
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,Fraud,0.0
4,1,2,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,No Fraud,0.0


# Online Payments Fraud Detection Model

In [11]:
# CLassification model

In [12]:
# Splitting the data

from sklearn.model_selection import train_test_split
x = np.array(data[["type","amount", "oldbalanceOrg", "newbalanceOrig"]])
y = np.array(data["isFraud"])

In [13]:
# Training the data

    # using Decision tree classifier
from sklearn.tree import DecisionTreeClassifier
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state = 42)

model = DecisionTreeClassifier()



In [19]:
# Classifying the transaction by feeding input

# Features = [type, amount, oldbalanceOrg, newbalanceOrig]
features = np.array([[2, 11668.14, 41554.0, 29885.86]])
print(model.predict(features))

['No Fraud']
